In [32]:
from keras import models
from keras import layers
from keras import optimizers
from keras import losses
from keras import metrics
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
import numpy as np
from keras.models import Sequential
from keras.layers import Dropout
from keras.callbacks import ModelCheckpoint

model = Sequential()

model.add(Conv2D(32, (5, 5), border_mode='same', input_shape=(128,128,3),data_format="channels_last", activation='relu'))
model.add(Conv2D(32, (5, 5), border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (5, 5), border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (5, 5), border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (5, 5), border_mode='same', activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256, activation='relu'))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1,activation='sigmoid'))
    
model.compile(loss=losses.binary_crossentropy,
            optimizer=optimizers.RMSprop(lr=0.0001),
            metrics=[metrics.binary_accuracy])

filepath="model_{epoch:02d}-{val_loss:.2f}.hdf5"

checkpoint = ModelCheckpoint(filepath, verbose=1, 
                            save_best_only=True)
model.load_weights('model_19-0.41.hdf5')

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), input_shape=(128, 128,..., data_format="channels_last", activation="relu", padding="same")`
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), activation="relu", padding="same")`
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), activation="relu", padding="same")`
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (5, 5), activation="relu", padding="same")`


In [33]:
import os
import random
import numpy as np
from PIL import Image
test_list = []
test_label = []

#generate label list
for root, dirs, files in os.walk('test_writing'):
    for file in files:
        if '.jpg' in file:
            test_list.append(os.path.join(root, file))
            test_label.append(1)
for root, dirs, files in os.walk('test_nowriting'):
    for file in files:
        if '.jpg' in file:
            test_list.append(os.path.join(root, file))
            test_label.append(0)

#random shuffle the video list          
c= list(zip(test_list,test_label))
random.shuffle(c)
test_list,test_label = zip(*c)

x_test = np.array([np.array(Image.open(im_dir).resize((128,128))) for im_dir in test_list])
y_test = np.array(test_label)

In [34]:
results=model.evaluate(x_test,y_test)

675/675 [==============================] - 9s 13ms/step


In [35]:
results

[0.715915310912662, 0.4681481420993805]

In [36]:
sample_1=[]
for root, dirs, files in os.walk('sample_frame1'):
    for file in files:
        if '.jpg' in file:
            sample_1.append(os.path.join(root, file))
sample_1_test = np.array([np.array(Image.open(im_dir).resize((128,128))) for im_dir in sample_1])

In [37]:
# for i in range(5):
import matplotlib.pyplot as plt
import jsonlines
prob = model.predict(sample_1_test)
pred = [int(p>0.5) for p in prob]
time = np.arange(15200,16000,5)/10
js = ""
for i, p in enumerate(prob):
    js += "({},{}) ".format(time[i], p)
with jsonlines.open("sample_1_test.jsonl",mode='w') as writer:
    writer.write(js)
plt.plot(time, pred)
plt.xlabel('time (s)')
plt.ylabel('prediction')
plt.savefig("sample_1_test.jpg")
plt.clf()

<Figure size 432x288 with 0 Axes>

In [38]:
frame_range=[[15200,16000],[17000,18000],[12000,13000],[11000,12000],[11000,12000]]

In [39]:
import matplotlib.pyplot as plt
import jsonlines
for num in range(1,6):
    sample=[]
    time = np.arange(frame_range[num-1][0],frame_range[num-1][1],5)/10
    for root, dirs, files in os.walk("sample_frame{}".format(num)):
        for file in files:
            if '.jpg' in file:
                sample.append(os.path.join(root, file))
    print(len(sample))
    sample_test = np.array([np.array(Image.open(im_dir).resize((128,128))) for im_dir in sample])
    prob = model.predict(sample_test)
    pred = [int(p>0.5) for p in prob]
    js = ""
    for i, p in enumerate(prob):
        js += "({},{}) ".format(time[i], p)
    with jsonlines.open("sample_{}_test.jsonl".format(i),mode='w') as writer:
        writer.write(js)
    plt.plot(time, pred)
    plt.xlabel('time (s)')
    plt.ylabel('prediction')
    plt.savefig("sample_{}_test.jpg".format(num))
    plt.clf()

160
200
200
200
200


<Figure size 432x288 with 0 Axes>

In [31]:
len(prob)

200